In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [2]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
train=pd.read_csv("/kaggle/input/playground-series-s5e7/train.csv")
test=pd.read_csv("/kaggle/input/playground-series-s5e7/test.csv")

In [3]:
numeric_cols_train = train.select_dtypes(include=np.number).columns
medians_train = train[numeric_cols_train].median()
train[numeric_cols_train] = train[numeric_cols_train].fillna(medians_train)
numeric_cols_test = test.select_dtypes(include=np.number).columns
medians_test = test[numeric_cols_test].median()
test[numeric_cols_test] = test[numeric_cols_test].fillna(medians_test)
le=LabelEncoder()
train['Stage_fear']=le.fit_transform(train['Stage_fear'])
train['Personality']=le.fit_transform(train['Personality'])
train['Drained_after_socializing']=le.fit_transform(train['Drained_after_socializing'])
test['Stage_fear']=le.fit_transform(test['Stage_fear'])
test['Drained_after_socializing']=le.fit_transform(test['Drained_after_socializing'])

In [4]:
from sklearn.model_selection import train_test_split
y=train['Personality']
x=train.drop('Personality',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.24)

In [5]:
from xgboost import XGBClassifier
params_xgb = {
    "n_estimators": [100, 200, 300],
    "max_depth": [1,3, 5, 7],
    "learning_rate": [0.001,0.01, 0.05],
    "subsample": [0.5, 0.7],
    "colsample_bytree": [0.6, 1.0]
}


In [6]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
xgb=XGBClassifier()
gridmodel=GridSearchCV(estimator=xgb,
                       param_grid=params_xgb,
                       cv=5,
                       n_jobs=-1,
                       verbose=2)
gridmodel.fit(x_train,y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] END colsample_bytree=0.6, learning_rate=0.001, max_depth=1, n_estimators=100, subsample=0.5; total time=   0.2s
[CV] END colsample_bytree=0.6, learning_rate=0.001, max_depth=1, n_estimators=100, subsample=0.5; total time=   0.1s
[CV] END colsample_bytree=0.6, learning_rate=0.001, max_depth=1, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.6, learning_rate=0.001, max_depth=1, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.6, learning_rate=0.001, max_depth=1, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.6, learning_rate=0.001, max_depth=1, n_estimators=200, subsample=0.5; total time=   0.2s
[CV] END colsample_bytree=0.6, learning_rate=0.001, max_depth=1, n_estimators=200, subsample=0.5; total time=   0.2s
[CV] END colsample_bytree=0.6, learning_rate=0.001, max_depth=1, n_estimators=200, subsample=0.7; total time=   0.2s
[

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 1.0],
                         'learning_rate': [0.001, 0.01, 0.05],
                         'max_depth': [1, 3, 5, 7],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.5, 0.7]},
             verbose=2)

In [7]:
bestparams=gridmodel.best_params_
bestmodel=XGBClassifier(**bestparams)
bestmodel.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [8]:
from sklearn.metrics import accuracy_score
y_pred = bestmodel.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy of best XGBoost model: {acc:.4f}")


Accuracy of best XGBoost model: 0.9681


In [9]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)
param_dist_rf = {
    'n_estimators': [80,100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 0.8, 1],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}
random_search_rf = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_dist_rf,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    random_state=42
)
random_search_rf.fit(x_train, y_train)



Fitting 5 folds for each of 20 candidates, totalling 100 fits

[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=300, subsample=0.5; total time=   0.6s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=300, subsample=0.7; total time=   0.6s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=7, n_estimators=100, subsample=0.5; total time=   0.2s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=7, n_estimators=100, subsample=0.5; total time=   0.2s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=7, n_estimators=100, subsample=0.7; total time=   0.3s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=7, n_estimators=200, subsample=0.5; total time=   0.5s
[CV] END 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 10, 20],
                                        'max_features': ['sqrt', 'log2', 0.8,
                                                         1],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [80, 100, 200]},
                   random_state=42, scoring='accuracy', verbose=2)

In [10]:
best_rf_params = random_search_rf.best_params_
bestrfmodel=RandomForestClassifier(**best_rf_params)
bestrfmodel.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=20, max_features=1,
                       min_samples_split=10)

In [11]:
y_pred_rf = bestrfmodel.predict(x_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy of best model: {acc_rf:.4f}")


Accuracy of best model: 0.9683


In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.base import clone
model1 = clone(bestmodel)     
model2 = clone(bestrfmodel)     
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
meta_features = []
meta_labels = []
for train_idx, val_idx in skf.split(x, y):
    x_train_fold, x_val_fold = x.iloc[train_idx], x.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    model1.fit(x_train_fold, y_train_fold)
    pred1 = model1.predict_proba(x_val_fold)[:, 1]
    model2.fit(x_train_fold, y_train_fold)
    pred2 = model2.predict_proba(x_val_fold)[:, 1]
    stacked = np.column_stack((pred1, pred2))
    meta_features.append(stacked)
    meta_labels.append(y_val_fold) 
x_meta = np.vstack(meta_features)
y_meta = np.hstack(meta_labels)
meta_model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42) 
meta_model.fit(x_meta, y_meta)
final_model1 = clone(bestmodel)
final_model2 = clone(bestrfmodel)
final_model1.fit(x, y)
final_model2.fit(x, y) 
pred1_test = final_model1.predict_proba(x_test)[:, 1]
pred2_test = final_model2.predict_proba(x_test)[:, 1]
x_test_meta = np.column_stack((pred1_test, pred2_test))
meta_preds = meta_model.predict_proba(x_test_meta)[:, 1]
meta_preds_labels = (meta_preds >= 0.5).astype(int)
acc = accuracy_score(y_test, meta_preds_labels)
roc = roc_auc_score(y_test, meta_preds)

print(f"✅ Stacking Accuracy: {acc:.4f}")
print(f"✅ Stacking ROC-AUC:  {roc:.4f}")


✅ Stacking Accuracy: 0.9699
✅ Stacking ROC-AUC:  0.9983


In [13]:
class EnsembleStacker:
    def __init__(self, base_model_1, base_model_2, meta_model):
        self.base_model_1 = clone(base_model_1)
        self.base_model_2 = clone(base_model_2)
        self.meta_model = clone(meta_model)

    def fit(self, x_train, y_train):
        self.base_model_1.fit(x_train, y_train)
        self.base_model_2.fit(x_train, y_train)
        pred1 = self.base_model_1.predict_proba(x_train)[:, 1]
        pred2 = self.base_model_2.predict_proba(x_train)[:, 1]
        stacked_input = np.column_stack((pred1, pred2))
        self.meta_model.fit(stacked_input, y_train)

    def predict(self, x_input):
        pred1 = self.base_model_1.predict_proba(x_input)[:, 1]
        pred2 = self.base_model_2.predict_proba(x_input)[:, 1]
        stacked_input = np.column_stack((pred1, pred2))
        meta_probs = self.meta_model.predict_proba(stacked_input)[:, 1]
        return (meta_probs >= 0.5).astype(int)
stacker = EnsembleStacker(bestmodel, bestrfmodel, meta_model)
stacker.fit(x, y)  
final_predictions = stacker.predict(test)  


In [14]:
mapping_dict = {0: 'Extrovert', 1: 'Introvert'}
prediction_string = np.array([mapping_dict[val] for val in final_predictions])
prediction_string

array(['Extrovert', 'Introvert', 'Extrovert', ..., 'Extrovert',
       'Extrovert', 'Introvert'], dtype='<U9')

In [15]:
submission=pd.read_csv('/kaggle/input/playground-series-s5e7/sample_submission.csv')
submission.head()


,id,Personality
0,18524,Extrovert
1,18525,Extrovert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Extrovert


In [16]:
submission['id']=test['id']
submission['Personality']=prediction_string
submission.to_csv("submission.csv", index=False)
